<b>Loading Libraries</b>

In [2]:
import time
import pprint
import glob
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
import string
import pickle
import gensim
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import stem_text
from nltk.tokenize import word_tokenize
from gensim.utils import tokenize
import numpy as np

<b>Preprocessing and tokenizing function</b>

In [22]:
def preprocess(text):
    return [word for word in word_tokenize(stem_text(strip_punctuation(text.lower().replace("\n", ""))))]

<b>Initializing global variables </b>

In [8]:
pp = pprint.PrettyPrinter(indent=2)
files = glob.glob('/home/rohan/Documents/Homework/WSTA/wiki-pages-text/*.txt')
files = [file for file in files if file != '/home/rohan/Documents/Homework/WSTA/wiki-pages-text/to_test.txt']
#files = ['/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-047.txt']
doc_dict = {}
file_dict = {}
df = pd.DataFrame()
stemmer = nltk.stem.PorterStemmer()
punc_table = str.maketrans({key: None for key in string.punctuation})

<b> Reading files into a dictionary </b>

In [10]:
start = time.time()
for file in files:
    with open(file, encoding = 'utf-8') as f:
        file_dict = {}
        print(file)
        for line in f:
            try:
                #print(line)
                line = line.split(maxsplit=2)
                #print(line)
                key = line[0] + "_" + line[1]
                value = line[2].rstrip('\n')
                file_dict[key] = value
                    #print(value)
            except Exception as e:
                print("ERROR",e, line)
        df = pd.DataFrame(file_dict.items(),columns=['doc_id','doc'])
        #print(df.head())
        df_doc = df['doc']
        print(df.shape)
        with open('/home/rohan/Documents/Homework/WSTA/wiki-processed/' + file.split('/')[-1], 'w') as fp:
            np.savetxt(fp, df_doc.values, fmt = '%s')   
        
        #print(file_dict)
print("Time taken: ", time.time() - start)

/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-047.txt
(252674, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-041.txt
(231331, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-107.txt
(210709, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-108.txt
(204263, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-098.txt
(230496, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-030.txt
(238710, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-031.txt
(235814, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-087.txt
(220828, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-077.txt
(236497, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-080.txt
(226107, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-010.txt
(225974, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-045.txt
(261898, 2)
/home/rohan/Documents/Homework/WSTA/wiki-pages-text/wiki-066.txt
(230106, 2)

<b> Loading into Dataframe </b>

In [25]:
df = pd.DataFrame(doc_dict.items(),columns=['doc_id','doc'])
del doc_dict

In [26]:
df_doc = df['doc']
df_doc.head()

0    Early elections for the nineteenth Knesset wer...
1    The International Mathematics & Design Associa...
2    Interstellar Low Ways is an album recorded by ...
3    Jack Daniel 's Racing may refer to either of t...
4    is a railway station operated by Meitetsu 's S...
Name: doc, dtype: object

In [27]:
with open('wiki_all_corpus.txt', 'w') as f:
    np.savetxt(f, df_doc.values, fmt = '%s')

<b> Pickle dataframe </b>

In [42]:
#pkl_name = file.split('/')[-1].split(".")[0]
df.to_pickle("wiki_47" + ".pickle")
del df

In [45]:
del df

<b> Load pickled dataframe </b>

In [3]:
df = pd.read_pickle('all_wiki.pickle')
df.shape

(5396106, 2)

In [4]:
df.head()

,doc_id,doc
0,"Israeli_legislative_election,_2013",Early elections for the nineteenth Knesset wer...
1,International_Mathematics_and_Design_Association,The International Mathematics & Design Associa...
2,Interstellar_Low_Ways,Interstellar Low Ways is an album recorded by ...
3,Jack_Daniel's_Racing,Jack Daniel 's Racing may refer to either of t...
4,Inba_Station,is a railway station operated by Meitetsu 's S...


In [ ]:
df['doc'] = df['doc'].apply(lambda x: list(tokenize(x)))

<b> Print dataframe and preprocess document</b>

In [9]:
start = time.time()
df_4['doc'] = df_4['doc'].apply(preprocess)
print("Number of documents:", len(df_4))
print("Time taken:", time.time() - start)

Number of documents: 1349026
Time taken: 1015.4041588306427


<b> Pickling preprocessed docs </b>

<b> Loading pickled processed dataframes </b>

In [3]:
#df_1 = pd.read_pickle('all_wiki_processed_1.pickle')
#df_2 = pd.read_pickle('all_wiki_processed_2.pickle')
#df_3 = pd.read_pickle('all_wiki_processed_3.pickle')
df_4 = pd.read_pickle('all_wiki_processed_4.pickle')

<b> Create Gensim dictionary </b>

In [4]:
start = time.time()
dictionary = gensim.corpora.Dictionary(df_4['doc'].tolist())
print(dictionary)
print("Time taken:", time.time() - start)

Dictionary(1674392 unique tokens: ['australian', 'brett', 'championshipperkin', 'championshiprichard', 'childress']...)
Time taken: 108.66674947738647


In [5]:
dictionary.save_as_text('dictionary_4')

<b> Creating BOW represention of documents </b>

In [17]:
start = time.time()
corpus = [dictionary.doc2bow(doc) for doc in df['doc_processed'].tolist()]
print(corpus[:5])
print("Time taken:", time.time() - start)

[[(0, 1), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 4), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 4), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 14), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1)], [(6, 1), (12, 1), (13, 1), (23, 1), (58, 1), (59, 1), (66, 3), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 1), (101, 1), (102, 1), (103, 1), (10

<b> Creating tf-idf model from this corpus </b>

In [18]:
start = time.time()
tf_idf = gensim.models.TfidfModel(corpus)
print(type(tf_idf))
s = 0
for i in corpus:
    s += len(i)
print(s)
print("Time taken:", time.time() - start)

<class 'gensim.models.tfidfmodel.TfidfModel'>
2893130
Time taken: 0.6749780178070068


<b> Creating Similarity index </b>

In [19]:
start = time.time()
sims = gensim.similarities.Similarity('/home/rohan/Documents/Homework/WSTA/work_dir',tf_idf[corpus], num_features=len(dictionary))
print("Time taken: ", time.time() - start)

Time taken:  19.05685067176819


<b> Getting query and showing documents related to the query </b>

In [ ]:
query = input("Enter statement: ")
start = time.time()
query_doc = preprocess(query)
query_doc_bow = dictionary.doc2bow(query_doc)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)
sim_list = sims[query_doc_tf_idf]
df['similarity_score'] = sim_list
print(type(sim_list), len(sim_list))
print(df.nlargest(5, ['similarity_score']))
#print("Position of most relevant doc: ", max_index)
print("Time taken:", time.time() - start)